In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import LASCO_catalog as lasco
import flare_persist as fp
import pandas as pd
import matplotlib as plt
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\sunpy\lightcurve\__init__.py:18: SunpyDeprecationWarning: As of v0.8.0, the `sunpy.lightcurve` module is deprecated and will be removed in a future version. Use `sunpy.timeseries` or `sunpy.map` for coordinate transformations.
  _SunpyDeprecationWarning)


In [3]:
#lasco_df = lasco.read_lasco_cat()
lasco_df = pd.read_csv("LASCO_cat.csv")

In [4]:
flare_df = pd.read_excel("ControlEvents_student.xls")
#flare_df = flare_df.iloc[4000:4199] 

In [5]:
flare_df = lasco.add_cme_width(flare_df,lasco_df)
flare_df = fp.flare_persistence(flare_df,12)

C:\Users\pedro.brea\SEPML\LASCO_catalog.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  flare_df['Width'][idx] = width
C:\ProgramData\Anaconda3\lib\site-packages\astropy\units\quantity.py:463: RuntimeWarning: invalid value encountered in less
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\astropy\units\quantity.py:463: RuntimeWarning: invalid value encountered in greater
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [6]:
#adding y label column indicating positive and negative SEP events - SEP events have Association = ProtonFlare
flare_df['sep'] = flare_df.Association.str.contains('^Proton').astype(int)

#convert the type II and type IV association to binary 
flare_df.TypeII = (flare_df.TypeII.str.lower() == "yes").astype(int)
flare_df.TypeIV = (flare_df.TypeIV.str.lower() == "yes").astype(int)

# Remove rows where optlocation = nan
flare_df = flare_df[flare_df.optlocation.astype('str') != 'nan']

#drop 7 samples where dfTrain FlrIntFlux2 is erroneously greater than 1000.0 (all control events)
flare_df = flare_df[flare_df.FlrIntFlux2 < 1000.0]

#replace all -1 cmespeed values with NaN
flare_df.cmespeed.replace(-1,np.nan, inplace = True)

In [ ]:
#flare_df = flare_df.dropna(how='any')

In [ ]:
flare_df.to_csv('ML_test.csv')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


#plt.subplot(3,3,1)
boxplt = flare_df.boxplot(column = 'FlrPeakFlux',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')


boxplt = flare_df.boxplot(column = 'Width',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
#plt.show()

boxplt = flare_df.boxplot(column = 'cmespeed',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()

boxplt = flare_df.boxplot(column = 'FlrIntFlux',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()

boxplt = flare_df.boxplot(column = 'FlrIntFlux2',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()

boxplt = flare_df.boxplot(column = 'emchianti',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()

boxplt = flare_df.boxplot(column = 'tchianti',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()

boxplt = flare_df.boxplot(column = 'FlarePersistence8',by = 'sep',layout = (1,1))
[ax_tmp.set_xlabel('SEP') for ax_tmp in np.asarray(boxplt).reshape(-1)]
fig = np.asarray(boxplt).reshape(-1)[0].get_figure()
fig.suptitle('')
plt.show()
